## Introduction
In this example, we show that how gear can be applied to find the bounds of error of a digital signal processing circuit throught the fixed-number lengths of the internal calculation.

The errors of computation of a digital signal processing circuit come from three sources: truncation error, quantization error, and inaccurate source.
1.  Truncation Error:
The truncation error is the error when a fixed-point number is converted from a longer representation to a shorter representation, which means the bits are not sufficient to represent the number, and thus the number must be truncated.
The truncation error occur when the result of an operation is connect to a signal with different word length.
2. Quantization Error:
The quantization error occurs when the an analog signal is converted into a digital signal, or an algorithm requires real numbers as its coefficients. As the fixed number cannot represent any real number with infinite precision. The number has to be quantized and thus are not as the exact value as the analog signal.

3. Inaccurate Source:
This error comes from the measurement in the analog signal because of noise and non-ideal effect on the sensor, and could be seen as a random noise on the analog signal.

The determination of the number of bits for fixed-point number requires tradeoff between the accuracy and the performance of the circuit.
Power, timing, and area of the circuit can benefit from reducing the number of bits for representation of fixed-point number as the gate count can be reduced, which results in shorter critical path length for optimization and smaller core footprint.
However, the accuracy of the signal processing decreases as more error is introduced in the computation and propagated to the output of the circuit.

Therefore, in digital signal processing circuit design. Exploration of the word lengths for all fixed-point number is a crucial stage from algorithm design to circuit implementation for optimizing the circuit performance while satisfying the system requirements.


## Preliminaries
#### Representation of Fixed-point Number

A word length of a fixed-point number can be described as a tuple $(n, p)$, where $n$ denotes the number of bits, and $p$ is the position of the binary point, with 0 position means the position before the most significant bit.

For example, A fixed-point number with word length $(5,2)$ with bit value "11011" means the binary number $11.011$, which is equivalent to $3.375$ in decimal.

First, we import some data structure for holding the wordlength, and the apis of gear.

In [2]:
from gear.terms.polyhedra.loaders import readContract, writeContract
from tool import PortWordLength
import numpy as np

In [3]:
# Definition of value_num method in PortWordLength class
#    def value_num(self):
#        return int(self.value, base=2) * 2 ** (- (self.n - self.p))


port = PortWordLength(n=5, p=2, value="11011")
print(f"The word length (n = {port.n}, p = {port.p}) of the bits \"{port.value}\" is: ")
print(port.value_num())

The word length (n = 5, p = 2) of the bits "11011" is: 
3.375


## Forming contract for capturing error of fixed-point number

Here we introduce how to form the contract for the fixed-point number operation.
First, we need to model how the error is generated.
Then we consider the propagation of the error.
We formulate the contract by combining the generation of error and propagation of the error.

#### Error Modeling
Each operation can be seen as a two-staged process.
First, an ideal output with certian wordlength is generated without losing any information, given the word lengths of all input ports.
Then the ideal output is truncated to the actual output port and could result in truncation error.
Therefore, the modeling of the ideal output wordlength and the truncation error is required to formulate the contract.

#### Truncation Error Modeling
In the following, we details how to model the truncation error.

##### Case 1. Binary Point Position Unchanged
In this case the change of word length only result in losing the lowest significant bits.
The maximal truncation error from the word length $(n_1, p)$ to the word length $(n_2, p)$ is $2^{p}(2^{-n_2}-2^{-n_1})$, when $n_1 > n_2$.
If $n_1 <= n_2$, there is no truncation and therefore the error is exactly $0$.

The following example shows how the error is bounded:

In [4]:
def truncation_error_same_position(pi, po):
    assert(pi.p == po.p)
    if pi.n > po.n:
        return 2 ** po.p * (2 ** (-po.n) - 2 ** (-pi.n))
    else :
        return 0

def truncate(pi, po):
    # separate the bits into two parts, before binary point and after binary point
    bits_before_point = pi.value[:pi.p]
    bits_after_point = pi.value[pi.p:]
    
    # truncate or appending 0 
    if po.p >= pi.p:
        bits_before_point = ("0" * (po.p - pi.p)) + bits_before_point
    else:
        bits_before_point = bits_before_point[pi.p - po.p:]

    if po.n - po.p >= pi.n - pi.p:
        bits_after_point = bits_after_point + "0" * (po.n - po.p - pi.n + pi.p)
    else:
        bits_after_point = bits_after_point[:(po.n - po.p)]

    # combine the both parts
    ret = bits_before_point + bits_after_point
    po.set_value(value = ret)
    return po.value

In [5]:
p1 = PortWordLength(n=7, p=2, name="p1", value = "1111111")
p2 = PortWordLength(n=5, p=2, name="p2")

print("Truncation Error: ")
print(truncation_error_same_position(p1, p2))
print("Example Value to produce the error")

truncate(p1, p2)
print(f"p1 = {p1.value_num()}, bits: {p1.value}")
print(f"p2 = {p2.value_num()}, bits: {p2.value}")
print(f"p1 - p2 = ", p1.value_num() - p2.value_num())

Truncation Error: 
0.09375
Example Value to produce the error
p1 = 3.96875, bits: 1111111
p2 = 3.875, bits: 11111
p1 - p2 =  0.09375


Then let's see what happen if the binary point position is changed
##### Case 2. Binary Point Position Change
Consider the case when $(n_1, p_1)$ to the word length $(n_2, p_2)$
If the binary point position is change and result in less bits before the binary point ($p_1 > p_2$), the significant bits in the initial representation will be lost. 
Therefore, it is reasonable to assume that we know exactly the maximum number of the initial number should not set bits that cannot be included in $2^{p_2}$, which sets an assumption that the initial number must be strictly less than $2^{p_2}$.
Then, we can see the fixed-point number as if the those uneccesary most significant bits ($p_1 - p_2$) can be discarded.
$(n_1, p_1)$ can be seen as $(n_1 - p_1 + p_2, p_2)$. 
Then it become the case one, only with the additional assumption.

The following example shows how the error can be calculated.

In [6]:
def truncation_error(pi, po):
    # remove uneccesary most significant bits
    # Remider: The assumption must hold!!
    pi_adjusted = PortWordLength(n=pi.n - pi.p + po.p, p=po.p)
    # return the truncation error as in case 1
    return truncation_error_same_position(pi=pi_adjusted, po=po)

def get_assumption_value(pi, po):
    if pi.p > po.p:
        return 2 ** po.p
    else:
        return float("inf") # no additional assumption needed

def check_assumption_value(pi, po):
    assumption_value = get_assumption_value(pi, po)
    print(f"Assumption: {pi.name} < {assumption_value}")
    if not pi.value_num() < assumption_value:
        print("Assumption failed, truncation of MSB occurs")
        return False
    else:
        print(f"Assumption Satisfied ({pi.name} = {pi.value_num()} < {assumption_value})")
        return True
        


In [7]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")

assert(check_assumption_value(pi=p1, po=p2))

print("Truncation Error: ")
print(truncation_error(p1, p2))
print("Example Value to produce the error")

truncate(p1, p2)
print(f"p1 = {p1.value_num()}, bits: {p1.value}")
print(f"p2 = {p2.value_num()}, bits: {p2.value}")
print(f"p1 - p2 = ", p1.value_num() - p2.value_num())

Assumption: p1 < 4
Assumption Satisfied (p1 = 2.4375 < 4)
Truncation Error: 
0.0625
Example Value to produce the error
p1 = 2.4375, bits: 0100111
p2 = 2.375, bits: 10011
p1 - p2 =  0.0625


#### Ideal Output Wordlength
After we analyze the truncation error, the next problem is what is the wordlength before truncation?
Each operation requires an minimal ideal output wordlength to fully hold the resulting value without losing any information.
We represent the ideal output wordlength using $(n^*, p^*)$.
This is the wordlength truncated before sending to the output port.
And $(n_1, p_1), (n_2, n_2)$ denotes the wordlength of two inputs.

The ideal output wordlength depends on the type of operation
Let's first examine the addition and then the multiplication

##### Addition
The following equations show the ideal output wordlnegth for addition.

$n^* = \max(n_1, n_2 - p_2 + p_1) + \min(0, p_1 - p_2) + 1$,

$p* = \max(p_1, p_2) + 1$.

The rationale behind the equations is to keep every bits in both inputs and add an additional one bits in the front of the output to handle the carry.

Using the ideal output wordlength and the above truncation error. We can calculate the error induced by addition

In [8]:
def compute_required_word_length_add(in1: PortWordLength, in2:PortWordLength) -> PortWordLength:
    new_n = max(in1.n, in2.n - in2.p + in1.p) - min(0, in1.p - in2.p) + 1
    new_p = max(in1.p, in2.p) + 1
    return PortWordLength(n=new_n, p=new_p)

def error_truncation_add(p1, p2, po):
    print(f"Input {p1.name}: (n={p1.n}, p={p1.p})")
    print(f"Input {p2.name}: (n={p2.n}, p={p2.p})")
    p_ideal = compute_required_word_length_add(in1=p1, in2=p2)
    print(f"Ideal Output: (n={p_ideal.n}, p={p_ideal.p})")
    print(f"Actual Output ({po.name}): (n={po.n}, p={po.p})")

    assumption_value = get_assumption_value(pi=p_ideal, po=po)
    print(f"Assumption: {p1.name} + {p2.name} < {assumption_value}")
    return truncation_error(pi=p_ideal, po=po)
    

In [9]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_truncation_add(p1=p1, p2=p2, po=p3)

print(f"Error of the addition calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=8, p=4)
Actual Output (p3): (n=5, p=3)
Assumption: p1 + p2 < 8
Error of the addition calculation: 0.1875



##### Multiplication
The following equations show the ideal output wordlnegth for multiplication.
$n^* = n_1 + n_2$,
$p* = p_1, p_2$.

In [10]:
def compute_required_word_length_mult(in1: PortWordLength, in2:PortWordLength) -> PortWordLength:
    new_n = in1.n + in2.n
    new_p = in1.p + in2.p
    return PortWordLength(n=new_n, p=new_p)

def error_truncation_mult(p1, p2, po):
    print(f"Input {p1.name}: (n={p1.n}, p={p1.p})")
    print(f"Input {p2.name}: (n={p2.n}, p={p2.p})")
    p_ideal = compute_required_word_length_mult(in1=p1, in2=p2)
    print(f"Ideal Output: (n={p_ideal.n}, p={p_ideal.p})")
    print(f"Actual Output ({po.name}): (n={po.n}, p={po.p})")

    assumption_value = get_assumption_value(pi=p_ideal, po=po)
    print(f"Assumption: {p1.name} * {p2.name} < {assumption_value}")
    return truncation_error(pi=p_ideal, po=po)

In [11]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_truncation_mult(p1=p1, p2=p2, po=p3)

print(f"Error of the multiplication calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=12, p=5)
Actual Output (p3): (n=5, p=3)
Assumption: p1 * p2 < 8
Error of the multiplication calculation: 0.2421875


#### Propagation of Error

The error $e$ already presents in a signal to the port $p$ will propagate to the final output.
Such error including error from the previus operation, quantization error and known inaccuracies of the input.
This error is independent to the error of truncation error, as truncation error compares the value of the output to the nominal value of the input signal.
As a result, the error can be represented as the sum of the propagation error and the truncation error in the stage: 

$e = e_p + e_t$,

where $e_p$ is the propagtion error from inputs regardless of how many bits we have in the output, and $e_t$ denotes the truncation error that occurs due to the descrepancy between the wordlength of the ideal output and that of the actual output.
Thus, it is important to explore differnet word length in the circuit, as one change to the word length affect the accuracy of the output.

Let's now see how the error is propagated in different operation

##### Propagation of Error in Addition
The error is exactly the sum of error from both inputs.
$e_p = e_1 + e_2$


In [12]:
def error_propagation_add(p1, p2, po):
    return p1.e + p2.e

In [13]:
p1 = PortWordLength(n=7, p=3, e=0.5, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, e=0.2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

e_p = error_propagation_add(p1=p1, p2=p2, po=p3)
e_t = error_truncation_add(p1=p1, p2=p2, po=p3)

err = e_p + e_t
print(f"Propagation error: {e_p}")
print(f"Truncation error: {e_t}")
print(f"Error bound of the addition calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=8, p=4)
Actual Output (p3): (n=5, p=3)
Assumption: p1 + p2 < 8
Propagation error: 0.7
Truncation error: 0.1875
Error bound of the addition calculation: 0.8875


##### Propagation of Error in Multiplication
The error is as follows:

$e_p = a_1*e_2 + a_2*e_1 - e_1 * e_2$, 

where $a_1$ is the maximal possible value from input $1$, $a_2$ is the maximum possible value from input $2$.


In [14]:
def get_actual_possible_value(in_port: PortWordLength):
    return (1 - (2 ** -in_port.n) )* 2 ** in_port.p

def error_propagation_mult(p1, p2, po):
    return p1.a * p2.e + p2.a * p1.e + p1.e * p2.e

In [15]:
p1 = PortWordLength(n=7, p=3, e=0.5, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, e=0.3, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

e_p = error_propagation_mult(p1=p1, p2=p2, po=p3)
e_t = error_truncation_mult(p1=p1, p2=p2, po=p3)

err = e_p + e_t
print(f"Propagation error: {e_p}")
print(f"Truncation error: {e_t}")
print(f"Error bound of the multiplication calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=12, p=5)
Actual Output (p3): (n=5, p=3)
Assumption: p1 * p2 < 8
Propagation error: 2.81875
Truncation error: 0.2421875
Error bound of the multiplication calculation: 3.0609375


##### General operation
Using the above analysis, we now formulate the contract for different operation.
The error of any general operations with arity n, denoted by $f(x_1, x_2,..., x_n)$ is as follows:

$e_p = f(x_1, x_2,..., x_n)) - f(x_1 - e_1, x_2 - e_2,..., x_n - e_n)$ 

#### Contract Formulation
Combining the error bound modeling from propagation and truncation. We can formalte contracts for each operation.
A contract is a pair of assumption and guarantee, denoted by $C = (A, G)$, where $C$ represents the contract, $A$ denotes its assumption, and $G$ is the guarantee.
In the following, we introduce how the contract is formulated for addition and multiplication.

##### Variables in the contract
To describe the behavior of the operations, we encode two variables for each port: the actual value and the error.
The actual value means the nominal value appears at the port instead of the ideal value we hope to use in the signal processing algorithm.
The error is the deviation of the nominal value to the ideal value in the signal processing algorithm.

##### Assumption
The only source of assumption is the assumption in truncation to prevent losing significant bits when the binary point positions of the ideal output and the truncated output is different.

##### Guarantee
The guarantee of the contract includees two part. The first part describes the error of the output port, and the second part ensures the range of the actual value in the output port.

The following functions show how we form contract for arbitrary length of input ports and output ports.
The "_a" variables are used as the variable for the actual value going through the port.

In [16]:
def form_contract_add(in_port1, in_port2, out_port):
    ret_contract = {}
    # define input/output vars
    ret_contract["InputVars"] = [f"{in_port1.name}_a", f"{in_port1.name}_e",
                                 f"{in_port2.name}_a", f"{in_port2.name}_e"]
    ret_contract["OutputVars"] = [f"{out_port.name}_a", f"{out_port.name}_e"]
    # get assumption
    ideal_out_port = compute_required_word_length_add(in1=in_port1, in2=in_port2)
    assumption_value = get_assumption_value(pi=ideal_out_port, po=out_port)
    # write assumption in the contract
    if assumption_value != float("inf"):
        ret_contract["assumptions"] =  [{"coefficients":{f"{in_port1.name}_a":1, f"{in_port2.name}_a":1},
                                    "constant":assumption_value}]
    else:
        ret_contract["assumptions"] = []
    # get guarantee
    e_t = truncation_error(pi=ideal_out_port, po=out_port)

    # write guarantee in the contract, note the propagation is encoded in the polyhedral constraints
    ret_contract["guarantees"] =  [{"coefficients":{f"{in_port1.name}_e":-1, f"{in_port2.name}_e":-1, f"{out_port.name}_e": 1},
                                "constant":e_t},
                                {"coefficients":{f"{out_port.name}_a": 1}, "constant":out_port.a},
                                {"coefficients":{f"{out_port.name}_a": 1}, "constant":in_port1.a + in_port2.a},
                                {"coefficients":{f"{out_port.name}_a": 1, f"{in_port1.name}_a": -1, f"{in_port2.name}_a": -1}, "constant":0}
                                ]
    return ret_contract


def form_contract_mult_const(in_port1, in_port_const, out_port):
    ret_contract = {}
    # define input/output vars
    ret_contract["InputVars"] = [f"{in_port1.name}_a", f"{in_port1.name}_e"]
    ret_contract["OutputVars"] = [f"{out_port.name}_a", f"{out_port.name}_e"]
    # get assumption
    ideal_out_port = compute_required_word_length_mult(in1=in_port1, in2=in_port_const)
    assumption_value = get_assumption_value(pi=ideal_out_port, po=out_port)
    # write assumption in the contract
    if assumption_value != float("inf"):
        print(assumption_value)
        ret_contract["assumptions"] =  [{"coefficients":{f"{in_port1.name}_a": in_port_const.a},
                                    "constant":assumption_value}]
    else:
        ret_contract["assumptions"] = []
    # get guarantee
    print(ideal_out_port.to_string())
    e_t = truncation_error(pi=ideal_out_port, po=out_port)

    # write guarantee in the contract, note the propagation is encoded in the polyhedral constraints
    ret_contract["guarantees"] =  [{"coefficients":
                                    {   f"{in_port1.name}_e":-in_port_const.a+in_port_const.e, 
                                        f"{in_port1.name}_a":-in_port_const.e,
                                        f"{out_port.name}_e": 1},
                                    "constant":e_t},
                                    {"coefficients":{f"{out_port.name}_a": 1}, "constant":out_port.a},
                                    {"coefficients":{f"{out_port.name}_a": 1}, "constant":in_port_const.a * in_port1.a},
                                    {"coefficients":{f"{out_port.name}_a": 1, f"{in_port1.name}_a": -in_port_const.a}, "constant":0}
                                    ]
    return ret_contract

In [16]:
p1 = PortWordLength(n=7, p=3, name="p1")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name="p3")
c1 = form_contract_add(in_port1=p1, in_port2=p2, out_port=p3)
print(c1)
contract1 = readContract(c1)
print(str(contract1))


p4 = PortWordLength(n=7, p=3, name="p4")
p5 = PortWordLength(n=5, p=2, e=0.03, value="11011", name="p5") # const
p6 = PortWordLength(n=5, p=3, name="p6")
c2 = form_contract_mult_const(in_port1=p4, in_port_const=p5, out_port=p6)
contract2 = readContract(c2)
print(str(contract2))

{'InputVars': ['p1_a', 'p1_e', 'p2_a', 'p2_e'], 'OutputVars': ['p3_a', 'p3_e'], 'assumptions': [{'coefficients': {'p1_a': 1, 'p2_a': 1}, 'constant': 8}], 'guarantees': [{'coefficients': {'p1_e': -1, 'p2_e': -1, 'p3_e': 1}, 'constant': 0.1875}, {'coefficients': {'p3_a': 1}, 'constant': 7.75}, {'coefficients': {'p3_a': 1}, 'constant': 11.8125}, {'coefficients': {'p3_a': 1, 'p1_a': -1, 'p2_a': -1}, 'constant': 0}]}
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>]
OutVars:[<Var p3_a>, <Var p3_e>]
A: 1*p1_a + 1*p2_a <= 8
G: -1*p1_e + -1*p2_e + 1*p3_e <= 0.1875, 1*p3_a <= 7.75, -1*p1_a + -1*p2_a + 1*p3_a <= 0.0
8
Port: , (n, p) = (12, 5), e = 0, a = 31.9921875
InVars: [<Var p4_a>, <Var p4_e>]
OutVars:[<Var p6_a>, <Var p6_e>]
A: 3.375*p4_a <= 8
G: -0.03*p4_a + -3.345*p4_e + 1.0*p6_e <= 0.2421875, 1.0*p6_a <= 7.75, -3.375*p4_a + 1.0*p6_a <= 0.0


#### Example 1
Consider the following simple system with only two adder.

![Example 1](figures/example1.png)

We will show that how gear can be applied to get the system errors.

First, we express each port using the PortWordLength class, and then form the contracts for the operations.

In [32]:
def create_example1():
    p1 = PortWordLength(n=5, p=2, name = "p1")
    p2 = PortWordLength(n=5, p=3, name = "p2")
    p3 = PortWordLength(n=5, p=3, name = "p3")
    c1 = form_contract_add(in_port1=p1, in_port2=p2, out_port=p3)

    p4 = PortWordLength(n=7, p=3, name = "p4")
    p5 = PortWordLength(n=6, p=3, name = "p5")
    c2 = form_contract_add(in_port1=p3, in_port2=p4, out_port=p5)

    contract1 = readContract(c1)
    contract2 = readContract(c2)
    print("Contract 1:\n" + str(contract1))
    print("Contract 2:\n" + str(contract2))
    return contract1, contract2, p1, p2, p3, p4, p5
    
contract1, contract2, p1, p2, p3, p4, p5 = create_example1()

Contract 1:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>]
OutVars:[<Var p3_a>, <Var p3_e>]
A: 1*p1_a + 1*p2_a <= 8
G: -1*p1_e + -1*p2_e + 1*p3_e <= 0.125, 1*p3_a <= 7.75, -1*p1_a + -1*p2_a + 1*p3_a <= 0.0
Contract 2:
InVars: [<Var p3_a>, <Var p3_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p3_a + 1*p4_a <= 8
G: -1*p3_e + -1*p4_e + 1*p5_e <= 0.0625, 1*p5_a <= 7.875, -1*p3_a + -1*p4_a + 1*p5_a <= 0.0


We then use the composition to get the system contract.

In [33]:
contract_sys = contract1.compose(contract2)
print("Contract Sys:\n" + str(contract_sys))

Contract Sys:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p4_a <= 0.250000000000000, 1*p1_a + 1*p2_a <= 8
G: -1*p1_e + -1*p2_e + -1*p4_e + 1*p5_e <= 0.187500000000000, -1*p4_a + 1*p5_a <= 7.75, -1*p1_a + -1*p2_a + -1*p4_a + 1*p5_a <= 0.0, 1*p5_a <= 7.875


The system contracts show the relation between the actual value and error bounds in input values.

If the designers know the context how the system is used, they can includes additional contracts to constrain the input:

First, we try an example that has no additional contraint on the input, meaning that it can take arbitrary value for the fixed-point number.

In [17]:
def form_contract_input(in_port):
    ret_contract = {}
    # define input/output vars
    ret_contract["InputVars"] = []
    ret_contract["OutputVars"] = [f"{in_port.name}_a", f"{in_port.name}_e"]
    # get assumption
    ret_contract["assumptions"] = []
    ret_contract["guarantees"] =  [ {"coefficients": {f"{in_port.name}_a": 1}, "constant": in_port.a},
                                    {"coefficients": {f"{in_port.name}_a": -1}, "constant": 0},
                                    {"coefficients": {f"{in_port.name}_e": 1}, "constant": in_port.e},
                                    {"coefficients": {f"{in_port.name}_e": -1}, "constant": -in_port.e}]
    return ret_contract

In [34]:
def test_example_1():
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = readContract(c_p1)
    contract_p2 = readContract(c_p2)
    contract_p4 = readContract(c_p4)
    
    try: 
        contract_sys = contract1.compose(contract2)
        contract_sys = contract_p1.compose(contract_sys)
        contract_sys = contract_p2.compose(contract_sys)
        contract_sys = contract_p4.compose(contract_sys)
        print("Contract Sys:\n" + str(contract_sys))
        return contract_sys
    except ValueError as e:
        print("Composition Error")
        print(e)

contract_sys = test_example_1()

Composition Error
The guarantees 
1*p2_a <= 7.75, -1*p2_a <= 0.0, 1*p2_e <= 0.0, -1*p2_e <= 0.0
were insufficient to abduce the assumptions 
1*p4_a <= 0.250000000000000, 1*p2_a <= 4.12500000000000
by eliminating the variables 
[<Var p2_a>, <Var p2_e>, <Var p5_a>, <Var p5_e>]


The result shows that the guarantees are not sufficient to abduce the assumption.

It can be observed easily that the input value of `p2_a` = 7.75 would violate the required assumption to prevent MSB from truncated.


Then we consider the case where certain constraints are known for the inputs:

In [35]:
p1.set_value("10000")
p2.set_value("01111")
p4.set_value("0000100")
contract_sys = test_example_1()

Contract Sys:
InVars: []
OutVars:[<Var p5_a>, <Var p5_e>]
A: true
G: 1*p5_a <= 6.00000000000000, 1*p5_e <= 0.187500000000000


Under these sets of constraints on the input, the system is compatible with the environment and thus we obtain the bounds on the actual output and the error bounds to an ideal output. 

The contract bring us the benefits of performing more operations on the system.

For example, we can apply the quotient to find word length for a certain signal.

Take the system as an example again. If we want to reduce the wordlength of the intermediate signal $P_3$, we can use quotient to get the required wordlength.

![Example 1](figures/example1.png)

To use quotient, we starts from the system contract.
The system contract describes what the system looks like without knowing the underlying components.
Let's say we can tolerate an error of the output by 0.1 given the same input constraints for the previous refined one.

The system contract should be like this:
```
A: True
G: 1*p5_a <= 6.00000000000000, 1*p5_e <= 0.1
```

We can invoke contract quotient to get the desired system contract:

In [38]:
def get_desired_system_contract():
    ret_contract = {}
    ret_contract["InputVars"] = []
    ret_contract["OutputVars"] = ["p5_a", "p5_e"]
    ret_contract["assumptions"] = []
    ret_contract["guarantees"] = [{"coefficients": {f"p5_a": 1}, "constant": 6.00},
                                  {"coefficients": {f"p5_e": 1}, "constant": 0.1}]
    return readContract(ret_contract)

contract_spec = get_desired_system_contract()
print(str(contract_spec))

InVars: []
OutVars:[<Var p5_a>, <Var p5_e>]
A: true
G: 1*p5_a <= 6.0, 1*p5_e <= 0.10000000000000009


In [39]:
def quotient_example1():
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = readContract(c_p1)
    contract_p2 = readContract(c_p2)
    contract_p4 = readContract(c_p4)

    tmp_c1 = contract_spec.quotient(contract_p1)
    tmp_c2 = tmp_c1.quotient(contract_p2)
    tmp_c3 = tmp_c2.quotient(contract_p4)

quotient_example1()


ValueError: not enough values to unpack (expected 2, got 1)

#### Example 2: Filter Design

We apply the contract to a simple filter that computes the moving average of the signal

$y[n] = 0.2 \times x[n-2] + 0.6 \times x[n-1] + 0.2 \times x[n] $

And the following function perform the filter on the signal

In [23]:

def example2_filter(x):
    v = np.array([0.2, 0.6, 0.2])
    return np.convolve(x, v)

print(example2_filter([1.25, 2.25, 3.7, 6.5, 1.5]))


[0.25 1.2  2.34 3.97 4.94 2.2  0.3 ]


If we want to implement this filter in a digital circuit in the transpose form, 

In [24]:
import math
def float_to_bin(x, word_length: PortWordLength):
    frac_part, int_part = math.modf(x)
    p = word_length.p
    # get integer part
    if int_part == 0:
        bin_str = ""
    else:
        bin_str = bin(int(int_part))[2:]
        if len(bin_str) > p:
            print("The port unable to hold the number, significant bits lost")
        else:
            bin_str = bin_str.zfill(p)
    # get fractional part
    frac_str = ""
    req_length = word_length.n - word_length.p
    while len(frac_str) != req_length:
        frac_part *= 2
        if frac_part >= 1:
            frac_str += '1'
            frac_part -= 1
        else:
            frac_str += '0'

    return bin_str + frac_str
    

In [25]:
in1 = PortWordLength(n=6, p=0, e=0, name="in1")
in2 = PortWordLength(n=6, p=0, e=0, name="in2")
in3 = PortWordLength(n=6, p=0, e=0, name="in3")
const1 = PortWordLength(n=6, p=0, name="const1")
const2 = PortWordLength(n=6, p=0, name="const2")
const3 = PortWordLength(n=6, p=0, name="const3")
mult_out1 = PortWordLength(n=6, p=0, name="mult_out1")
mult_out2 = PortWordLength(n=6, p=0, name="mult_out2")
mult_out3 = PortWordLength(n=6, p=0, name="mult_out3")
add_out1 = PortWordLength(n=6, p=0, name="add_out1")
add_out2 = PortWordLength(n=6, p=0, name="add_out2")

const1.set_value(float_to_bin(0.2, const1))
const2.set_value(float_to_bin(0.6, const2))
const3.set_value(float_to_bin(0.2, const3))
const1.set_e(0.2 - const1.value_num())
const2.set_e(0.6 - const2.value_num())
const3.set_e(0.2 - const3.value_num())

c1 = form_contract_mult_const(in_port1=in1, in_port_const=const1, out_port=mult_out1)
c2 = form_contract_mult_const(in_port1=in2, in_port_const=const2, out_port=mult_out2)
c3 = form_contract_mult_const(in_port1=in3, in_port_const=const3, out_port=mult_out3)

ci1 = form_contract_input(in_port=in1)
ci2 = form_contract_input(in_port=in2)
ci3 = form_contract_input(in_port=in3)

contract1 = readContract(c1)
contract2 = readContract(c2)
contract3 = readContract(c3)
print(str(contract1))
contract_i1 = readContract(ci1)
contract_i2 = readContract(ci2)
contract_i3 = readContract(ci3)
# print(str(contract1))
# print(str(contract2))
# print(str(contract3))
c4 = form_contract_add(in_port1 = mult_out1, in_port2=mult_out2, out_port=add_out1)
c5 = form_contract_add(in_port1 = add_out1, in_port2=mult_out3, out_port=add_out2)
contract4 = readContract(c4)
contract5 = readContract(c5)
# print(str(contract4))
# print(str(contract5))
contract_system = contract1.compose(contract2)
contract_system = contract_system.compose(contract3)
contract_system = contract_system.compose(contract4)
contract_system = contract_system.compose(contract5)
print(str(contract_system))
contract_system = contract_i1.compose(contract_system)
print(str(contract_i1))
print(str(contract_i2))
print(str(contract_i3))
print(str(contract_system))
contract_system = contract_i2.compose(contract_system)
contract_system = contract_i3.compose(contract_system)
print(str(contract_system))


Port: , (n, p) = (12, 0), e = 0, a = 0.999755859375
Port: , (n, p) = (12, 0), e = 0, a = 0.999755859375
Port: , (n, p) = (12, 0), e = 0, a = 0.999755859375
InVars: [<Var in1_a>, <Var in1_e>]
OutVars:[<Var mult_out1_a>, <Var mult_out1_e>]
A: true
G: -0.012500000000000011*in1_a + -0.175*in1_e + 1.0*mult_out1_e <= 0.015380859375, 1.0*mult_out1_a <= 0.1845703125, -0.1875*in1_a + 1.0*mult_out1_a <= 0.0
InVars: [<Var in1_a>, <Var in1_e>, <Var in2_a>, <Var in2_e>, <Var in3_a>, <Var in3_e>]
OutVars:[<Var add_out2_a>, <Var add_out2_e>]
A: true
G: 1*add_out2_e + -0.0125000000000000*in1_a + -0.175000000000000*in1_e + -0.00625000000000000*in2_a + -0.587500000000000*in2_e + -0.0125000000000000*in3_a + -0.175000000000000*in3_e <= 0.0461425781250000, 1*add_out2_a <= 0.953613281250000
InVars: []
OutVars:[<Var in1_a>, <Var in1_e>]
A: true
G: 1*in1_a <= 0.984375, -1*in1_a <= 0.0, 1*in1_e <= 0.0, -1*in1_e <= 0.0
InVars: []
OutVars:[<Var in2_a>, <Var in2_e>]
A: true
G: 1*in2_a <= 0.984375, -1*in2_a <= 0.0

The result shows that we can get an upper bound on the error of the output `add_out_e` is bounded by about 0.07690.
And this is the extreme case when the input is the following case:

Note that this is always an upper bound to the actual errors as we abstract the calculation and considers the worse case truncation error for each operation.

If the obtained error is smaller than the specification, we can rest assure that the design already satisfy the goal and we can either submit the design or change some wordlength and see if we can use smaller wordlength to achieve the same specification.

If the obtained error is larger than the specification, the level of abstraction is not sufficient to prove the correctness of the design.
Refinement of the error model or extensive verfication is therefore needed to verify the design.
 

To see the actual errors for this specific case, we can enumerate all input sequence.


In [26]:
for i1 in range(0, 2**6):
    in1_a = float_to_bin(i1/ 2**6, in1)
    in1.set_value(in1_a)
    for i2 in range(0, 2**6):
        in2_a = float_to_bin(i2/ 2**6, in2)
        in2.set_value(in2_a)
        for i3 in range(0, 2**6):
            in3_a = float_to_bin(i3/2**6, in3)
            in3.set_value(in3_a)


            in1.value_num() * const1.value_num()
            in2.value_num() * const2.value_num()
            in3.value_num() * const3.value_num()

    
            


In [27]:
bin(int(0.984375*2**6) * int(0.984375*2**6))

'0b111110000001'

In [28]:
# exp
c1 = {'InputVars': [], 
'OutputVars': ['p2_a', 't2'], 
'assumptions': [], 
'guarantees': [ {'coefficients': {'p2_a': 1}, 'constant': 0.1}, 
                {'coefficients': {'p2_a': -1}, 'constant': 0.0},
                {'coefficients': {'t2': 1}, 'constant': 0.1}, 
                {'coefficients': {'t2': -1}, 'constant': 0.0}
              ]}

c2 = {'InputVars': ['p2_a', 't2'], 
'OutputVars': ['p3_e'], 
'assumptions': [], 
'guarantees': [{'coefficients': {'p3_e': 1, 'p2_a': 1, 't2':-1}, 'constant': 10}]}

contract1 = readContract(c1)
contract2 = readContract(c2)
system = contract1.compose(contract2)
print(str(system))



InVars: []
OutVars:[<Var p3_e>]
A: true
G: 1*p3_e <= 10.1000000000000


#### Example 3: Fast-Fourier Transform (FFT) Cooley-Tucker Algorthm

We applied the contract on the FFT Cooley-Tucker Algorithm of the Radix 2 decimation-in-time form.
The following function implements the eight-point FFT.

In [29]:
import numpy as np

def fft_8points(x):

    s1 = np.zeros(x.shape, dtype=complex)
    W0 = np.exp(-2j * np.pi/8 * 0)
    W1 = np.exp(-2j * np.pi/8 * 1)
    W2 = np.exp(-2j * np.pi/8 * 2)
    W3 = np.exp(-2j * np.pi/8 * 3)
    #print(W0, W1, W2, W3)

    s1[0] = x[0] + W0 * x[4]
    s1[1] = x[0] - W0 * x[4]
    s1[2] = x[2] + W0 * x[6]
    s1[3] = x[2] - W0 * x[6]
    s1[4] = x[1] + W0 * x[5]
    s1[5] = x[1] - W0 * x[5]
    s1[6] = x[3] + W0 * x[7]
    s1[7] = x[3] - W0 * x[7]

    s2 = np.zeros(x.shape, dtype=complex)
    s2[0] = s1[0] + W0 * s1[2]
    s2[2] = s1[0] - W0 * s1[2]
    s2[1] = s1[1] + W2 * s1[3]
    s2[3] = s1[1] - W2 * s1[3]
    s2[4] = s1[4] + W0 * s1[6]
    s2[6] = s1[4] - W0 * s1[6]
    s2[5] = s1[5] + W2 * s1[7]
    s2[7] = s1[5] - W2 * s1[7]

    s3 = np.zeros(x.shape, dtype=complex)
    s3[0] = s2[0] + W0 * s2[4]
    s3[4] = s2[0] - W0 * s2[4]
    s3[1] = s2[1] + W1 * s2[5]
    s3[5] = s2[1] - W1 * s2[5]
    s3[2] = s2[2] + W2 * s2[6]
    s3[6] = s2[2] - W2 * s2[6]
    s3[3] = s2[3] + W3 * s2[7]
    s3[7] = s2[3] - W3 * s2[7]

    return s3

t = np.arange(8)
#x = np.sin(t * 2 * np.pi / 8)

x = np.array([1,1,1,1,1,1,1,2], dtype=complex)
print(x)
print(fft_8points(x=x))
print(np.fft.fft(x))

x = np.array([1,0], dtype=complex)
print(np.fft.fft(x))



[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 2.+0.j]
[ 9.00000000e+00+0.j          7.07106781e-01+0.70710678j
 -6.12323400e-17+1.j         -7.07106781e-01+0.70710678j
 -1.00000000e+00+0.j         -7.07106781e-01-0.70710678j
  6.12323400e-17-1.j          7.07106781e-01-0.70710678j]
[ 9.        +0.j          0.70710678+0.70710678j  0.        +1.j
 -0.70710678+0.70710678j -1.        +0.j         -0.70710678-0.70710678j
  0.        -1.j          0.70710678-0.70710678j]
[1.+0.j 1.+0.j]


In [30]:
p4 = PortWordLength(n=8, p=3, name="p4")
p5 = PortWordLength(n=8, p=3, e=0.03, value="11011", name="p5") # const
p6 = PortWordLength(n=8, p=3, name="p6")

AssertionError: 

### Problem with Composition Order
The order matters and create too restrictive bounds.

In [18]:
def create_example1_by_p3(p3_n, p3_p):
    p1 = PortWordLength(n=5, p=2, name = "p1")
    p2 = PortWordLength(n=5, p=3, name = "p2")
    p3 = PortWordLength(n=p3_n, p=p3_p, name = "p3")
    c1 = form_contract_add(in_port1=p1, in_port2=p2, out_port=p3)

    p4 = PortWordLength(n=7, p=3, name = "p4")
    p5 = PortWordLength(n=6, p=3, name = "p5")
    c2 = form_contract_add(in_port1=p3, in_port2=p4, out_port=p5)

    p1.set_value("10000")
    p2.set_value("00001")
    p4.set_value("0001100")

    contract1 = readContract(c1)
    contract2 = readContract(c2)
    #print("Contract 1:\n" + str(contract1))
    #print("Contract 2:\n" + str(contract2))
    return contract1, contract2, p1, p2, p3, p4, p5

def compose_order1(contract1, contract2, p1, p2, p3, p4, p5):
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = readContract(c_p1)
    contract_p2 = readContract(c_p2)
    contract_p4 = readContract(c_p4)
    
    try: 
        contract_sys = contract_p1.compose(contract1)
        contract_sys = contract_p2.compose(contract_sys)
        contract_sys = contract_sys.compose(contract2)
        contract_sys = contract_p4.compose(contract_sys)
        print("Contract Sys:\n" + str(contract_sys))
        return contract_sys
    except ValueError as e:
        print("Composition Error")
        print(e)
        raise ValueError("Composition Fails")

def compose_order2(contract1, contract2, p1, p2, p3, p4, p5):
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = readContract(c_p1)
    contract_p2 = readContract(c_p2)
    contract_p4 = readContract(c_p4)
    
    try: 
        contract_sys = contract1.compose(contract2)
        contract_sys = contract_p2.compose(contract_sys)
        contract_sys = contract_sys.compose(contract2)
        contract_sys = contract_p4.compose(contract_sys)
        print("Contract Sys:\n" + str(contract_sys))
        return contract_sys
    except ValueError as e:
        print("Composition Error")
        print(e)
        raise ValueError("Composition Fails")

contract1, contract2, p1, p2, p3, p4, p5 = create_example1_by_p3(p3_n=5, p3_p=3)
contract_sys = compose_order1(contract1, contract2, p1, p2, p3, p4, p5)
print(str(contract_sys))

Contract Sys:
InVars: []
OutVars:[<Var p5_a>, <Var p5_e>]
A: true
G: 1*p5_a <= 3.00000000000000, 1*p5_e <= 0.187500000000000
InVars: []
OutVars:[<Var p5_a>, <Var p5_e>]
A: true
G: 1*p5_a <= 3.00000000000000, 1*p5_e <= 0.187500000000000
